In [2]:
import sys
import os
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs4
from lxml import html

from tqdm.notebook import tqdm

In [3]:
# チェック用ASINリスト
df = pd.read_csv('./登録管理.csv')

In [25]:
# df0104 = pd.DataFrame([],columns=['ASIN',])
# for product_code in tqdm(df['code']):
#     asin_code = product_code[:-4].upper()
#     dic = {'ASIN': asin_code}
#     df_ = pd.DataFrame(dic, index=[1])
#     df0104 = pd.concat([df0104, df_]).reset_index(drop=True)

  0%|          | 0/246291 [00:00<?, ?it/s]

In [29]:
# df0115 = df0104.head(100)
# df0115.to_csv('test_asin.csv', index=False, encoding='cp932')

In [3]:
# # チェック用URLリスト
# check_url_list = [
# 'B015CELJBM',
# 'B017R5S3OU',
# 'B01B12S0ES',
# 'B01B12S0QQ',
# 'B01BGMUMXA',
# 'B01BGMUSOS',
# 'B01G56AWSC',
# 'B01G56B05Q',
# 'B01G56B1IC',
# 'B01G56B678',
# 'B01K1QQH2W',
# ]

In [4]:
# asin_code = 'B017R5S3OU'

In [5]:
# このファイルと同じディレクトリに.wdmフォルダを作成し、
# そこにchromedriverが保存されるようにする
os.environ['WDM_LOCAL'] = '1'
# ------------------------------

In [6]:
# === Options ===
options = Options()
options.set_capability("pageLoadStrategy", "eager") # HTMLを読み込み次第スクレイピングを実行させる
# options.add_argument('--headless') # ヘッドレスモード

In [7]:
# # 対象アクセス
# driver = webdriver.Chrome(
#     service=Service(ChromeDriverManager().install()),
#     options=options)
# driver.implicitly_wait(10)
# driver.get(f'https://www.amazon.co.jp/gp/product/{asin_code}')

In [8]:
"""
在庫切れ確認
"""

# 在庫切れチェック用X_Pathのリスト
soled_xpath_list = [
    '/html/body/div[2]/div[3]/div[3]/div[6]/div[1]/div[4]/div/div/div/form/div/div/div[1]/span[1]',
    '/html/body/div[2]/div[2]/div[6]/div[5]/div[1]/div[3]/div/div/div/form/div/div/div[1]/span[1]',
    '/html/body/div[2]/div[3]/div[6]/div[5]/div[1]/div[3]/div/div/div/form/div/div/div[1]/span[1]',
    '/html/body/div[2]/div[2]/div[7]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[3]/div/div[4]/div/div[1]/span[1]',
    '/html/body/div[2]/div[3]/div[7]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[3]/div/div[4]/div/div[1]/span[1]',
    
]

def soled_check(driver, soled_xpath_list):
    """
    在庫切れチェック
    '在庫切れ'の文字列がある場合はTure
    '在庫切れ'の文字列がない場合はFalse
    
    商品ページに'在庫切れ'がある時点でループを終了する為
    ループ内で文字列チェック
    """
    
    # seleniumからbs4に移行
    data = driver.page_source.encode('utf-8')
    soup_parsed_data  = bs4(data, 'html.parser')
    lxml_coverted_data = html.fromstring(str(soup_parsed_data))
    
    # === 在庫切れテキストオブジェクト取得ループ ===
    for soled_xpath in soled_xpath_list:
        elem_list = lxml_coverted_data.xpath(soled_xpath)
        
        # オブジェクトがあるか確認
        if elem_list != []:
            soled_text = elem_list[0].text
            print(soled_text)
            
            # 文字列確認
            if '在庫切れ' in soled_text:
                soled_check_flag = True
                # ループ終了
                break
                
            else:
                soled_check_flag = False
                
        # オブジェクトがない場合
        else:
            soled_check_flag = False
            
    return soled_check_flag

In [9]:
"""
中古の確認
"""

# 中古チェック用X_Pathのリスト
old_xpath_list = [
    '/html/body/div[2]/div[2]/div[6]/div[5]/div[1]/div[3]/div/div/div/form/div/div[1]/div/div/div/div[1]/div[1]/div/span[1]',
    
]

def old_check(driver, old_xpath_list):
    """
    中古チェック
    '中古'の文字列がある場合はTure
    '中古'の文字列がない場合はFalse
    
    商品ページに'中古'がある時点でループを終了する為
    ループ内で文字列チェック
    """
    
    # seleniumからbs4に移行
    data = driver.page_source.encode('utf-8')
    soup_parsed_data  = bs4(data, 'html.parser')
    lxml_coverted_data = html.fromstring(str(soup_parsed_data))
    
    # === 中古品テキストオブジェクト取得ループ ===
    for old_xpath in old_xpath_list:
        elem_list = lxml_coverted_data.xpath(old_xpath)
        
        # オブジェクトがあるか確認
        if elem_list != []:
            old_text = elem_list[0].text
            print(old_text)
            
            # 文字列確認
            if '中古' in old_text:
                old_check_flag = True
                # ループ終了
                break
                
            else:
                old_check_flag = False
                
        # オブジェクトがない場合
        else:
            old_check_flag = False
    
    return old_check_flag

In [10]:
"""
出荷元の確認
"""

# 出荷元チェック用X_Pathのリスト
shipper_xpath_list = [
    '/html/body/div[3]/div[2]/div[6]/div[5]/div[1]/div[3]/div/div/div/div/form/div/div/div/div/div[3]/div/div[18]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[3]/div[3]/div[6]/div[1]/div[4]/div/div/div[1]/div/div/div[1]/div/div[2]/div/form/div/div/div[18]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[2]/div[6]/div[5]/div[1]/div[3]/div/div/div/div/form/div/div/div/div/div[3]/div/div[18]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[3]/div[3]/div[6]/div[1]/div[4]/div/div/div/div/form/div/div/div/div/div[3]/div/div[18]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[3]/div[6]/div[5]/div[1]/div[3]/div/div/div/div/form/div/div/div/div/div[3]/div/div[18]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[3]/div[7]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[3]/div/div[17]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[2]/div[7]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[3]/div/div[17]/div[1]/div[1]/div[2]/div/span',
    '/html/body/div[2]/div[3]/div[6]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[3]/div/div[17]/div[1]/div[1]/div[2]/div/span',
    
]

def shipper_check(driver, shipper_xpath_list):
    """
    出荷元チェック
    出荷元がNG(Amazon以外)の場合はTure
    出荷元がOK(Amazon)の場合はFalse
    
    ループ外で文字列チェック
    """
    
    # seleniumからbs4に移行
    data = driver.page_source.encode('utf-8')
    soup_parsed_data  = bs4(data, 'html.parser')
    lxml_coverted_data = html.fromstring(str(soup_parsed_data))
    
    # === while elem_list_len == 0:のループ設定値 ===
    elem_list_len = 0
    i = 0
    
    # === 出荷元のオブジェクト取得ループ ===
    while elem_list_len == 0:
        
        # === 例外処理 ===
        if i >= len(shipper_xpath_list):
            print(f'{sys._getframe().f_code.co_name} 未設定X_Pathエラー')
            sys.exit()
            
        elem_list = lxml_coverted_data.xpath(shipper_xpath_list[i])
        
        # === ループ設定値の変更 ===
        elem_list_len = len(elem_list)
        i += 1
        
    # === 出荷元の不要文字列削除 ===
    product_shipper = elem_list[0].text
    print(product_shipper)
    product_shipper = re.sub(r'\s+', '', product_shipper)

    # === 出荷元確認 ===
    if 'Amazon' != product_shipper and 'Amazon.co.jp' != product_shipper and \
    'アマゾン' != product_shipper and 'アマゾン合同会社' != product_shipper:
        shipper_check_flag = True
    else:
        shipper_check_flag = False
    
    return shipper_check_flag

In [11]:
"""
価格取得
"""

# 価格取得用X_Pathのリスト
price_xpath_list = [
    '/html/body/div[2]/div[3]/div[3]/div[6]/div[1]/div[4]/div/div/div/div/form/div/div/div/div/div[2]/div[1]/div/span/span[2]/span[2]',
    '/html/body/div[2]/div[2]/div[6]/div[5]/div[1]/div[3]/div/div/div/div/form/div/div/div/div/div[2]/div[1]/div/span/span[2]/span[2]',
    '/html/body/div[2]/div[3]/div[3]/div[6]/div[1]/div[4]/div/div/div[1]/div/div/div[1]/div/div[1]/h5/div[2]/div[1]/div/div/span/span[2]/span[2]',
    '/html/body/div[2]/div[3]/div[6]/div[5]/div[1]/div[3]/div/div/div/div/form/div/div/div/div/div[2]/div[1]/div/span/span[2]/span[2]',
    '/html/body/div[2]/div[3]/div[3]/div[6]/div[1]/div[4]/div/div/div/form/div/div/div[1]/span[1]',
    '/html/body/div[2]/div[2]/div[6]/div[5]/div[1]/div[3]/div/div/div/form/div/div/div[1]/span[1]',
    '/html/body/div[2]/div[3]/div[7]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[2]/div[1]/div/span/span[2]/span[2]',
    '/html/body/div[2]/div[2]/div[7]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[2]/div[1]/div/span/span[2]/span[2]',
    '/html/body/div[2]/div[3]/div[6]/div[4]/div[1]/div[2]/div/div/div/div/form/div/div/div/div/div[2]/div[1]/div/span/span[2]/span[2]',
    
]

def price_get(driver, price_xpath_list):
    """
    価格取得
    ループ外で価格取得
    """
    
    # seleniumからbs4に移行
    data = driver.page_source.encode('utf-8')
    soup_parsed_data  = bs4(data, 'html.parser')
    lxml_coverted_data = html.fromstring(str(soup_parsed_data))
    
    # === while elem_list_len == 0:のループ設定値 ===
    elem_list_len = 0
    i = 0
    
    # === 価格のオブジェクト取得ループ ===
    while elem_list_len == 0:
        
        # === 例外処理 ===
        if i >= len(price_xpath_list):
            print(f'{sys._getframe().f_code.co_name} 未設定X_Pathエラー')
            sys.exit()
            
        elem_list = lxml_coverted_data.xpath(price_xpath_list[i])
        
        # === ループ設定値の変更 ===
        elem_list_len = len(elem_list)
        i += 1
        
    # === 価格の不要文字列削除 ===
    product_price = elem_list[0].text
    print(product_price)
    product_price = re.sub(r'\D', '', product_price)
    
    return product_price

In [12]:
sleep_time = 0.1

for product_code in tqdm(df['code']):
    print(f'==={product_code}===')
    asin_code = product_code[:-4].upper()
    
    # 対象アクセス
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options)
    driver.implicitly_wait(10)
    driver.get(f'https://www.amazon.co.jp/gp/product/{asin_code}')
    
    # 在庫切れフラグチェック
    soled_check_flag = soled_check(driver, soled_xpath_list)
    if soled_check_flag:
        print(f'在庫切れです：{driver.current_url}')
        print(f'=============')
        driver.quit()
        time.sleep(sleep_time)
        continue
    
    # 中古フラグチェック
    old_check_flag = old_check(driver, old_xpath_list)
    if old_check_flag:
        print(f'中古です：{driver.current_url}')
        print(f'=============')
        driver.quit()
        time.sleep(sleep_time)
        continue
        
    # 出荷元フラグチェック
    shipper_check_flag = shipper_check(driver, shipper_xpath_list)
    if shipper_check_flag:
        print(f'出荷元NG：{driver.current_url}')
        print(f'=============')
        driver.quit()
        time.sleep(sleep_time)
        continue
        
    # 価格取得
    product_price = price_get(driver, price_xpath_list)
    if product_price == '':
        print(f'空です：{driver.current_url}')
        print(f'=============')
        driver.quit()
        time.sleep(sleep_time)
        continue
        
    print(product_price)
    print(f'=============')
    driver.quit()
    time.sleep(sleep_time)

  0%|          | 0/246291 [00:00<?, ?it/s]

===b00569k9omkk00===
    一時的に在庫切れ; 入荷時期は未定です。   
在庫切れです：https://www.amazon.co.jp/gp/product/B00569K9OM
===b00569ki4skk00===
    一時的に在庫切れ; 入荷時期は未定です。   
在庫切れです：https://www.amazon.co.jp/gp/product/B00569KI4S
===b00569knqgkk00===
Amazon.co.jp
6,639
6639
===b00569kpkukk00===
    一時的に在庫切れ; 入荷時期は未定です。   
在庫切れです：https://www.amazon.co.jp/gp/product/B00569KPKU
===b0147yovhwkk00===
 残り1点 ご注文はお早めに 
Amazon.co.jp
1,451
1451
===b00me65hvkkk00===
Amazon
2,305
2305
===b00rwn0pvwkk00===
Amazon
3,735
3735
===b00x94sooskk00===
Amazon
3,735
3735
===b00ybiqix0kk00===
Amazon
2,305
2305
===b011nbatn2kk00===
Amazon
2,305
2305
===b019gy5izckk00===
現在在庫切れです。
在庫切れです：https://www.amazon.co.jp/gp/product/B019GY5IZC
===b01c27x3fckk00===
現在在庫切れです。
在庫切れです：https://www.amazon.co.jp/gp/product/B01C27X3FC
===b01fxba4n8kk00===
Amazon
2,395
2395
===b01fxbzzkukk00===
Amazon
2,305
2305
===b01iqhuizkkk00===
Amazon
2,995
2995
===b01lwsbvl0kk00===
Amazon
2,305
2305
===b01lzdyd8hkk00===
Amazon
3,295
3295
===b01mtsasy1kk00===
Amaz

SystemExit: 

C:\Users\user\hoges\.hoge1\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
STOP

In [13]:
driver.quit()

In [ ]:
for asin_code in check_url_list:
    # 対象アクセス
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options)
    driver.implicitly_wait(10)
    driver.get(f'https://www.amazon.co.jp/gp/product/{asin_code}')
    
    print(f'====={asin_code}====')

    time_start = time.time()
    shipper_check(driver, shipper_xpath_list)
    time_end = time.time()
    print(time_end-time_start)

    print('=====================')

    time_start = time.time()
    shipper_check_test(driver, shipper_xpath_list)
    time_end = time.time()
    print(time_end-time_start)